In [2]:
import matplotlib.pyplot as plt
import matplotlib
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import random
import torchvision
import pretrainedmodels

matplotlib.style.use('ggplot')

from sklearn.model_selection import train_test_split
from torchvision.transforms import transforms
from torch.utils.data import DataLoader, Dataset
from torchvision import models, datasets
from torchvision.datasets import ImageFolder

def seed_everything(SEED=41):
    random.seed(SEED)
    np.random.seed(SEED)
    torch.manual_seed(SEED)
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)

SEED=42
seed_everything(SEED=SEED)

In [3]:
class_list =[
    ['acura',173250000],
    ['alfaromeo',82500000],
    ['buick',59400000],
    ['cadillac',110000000],
    ['dodge',88000000],
    ['fiat',33000000],
    ['opel',44000000]
]

In [4]:
print('test')

test


In [5]:
dataset_path = "F:\프로젝트\CarLogoDetection\data"
train_val_split_ratio = 0.8
learning_rate = 0.0001
num_epoch = 100
batch_size = 8
class_num = 7

trans = transforms.Compose([
    transforms.Resize((224, 224)),  # 이미지 크기 조정
    transforms.ToTensor(),  # 이미지를 텐서로 변환
])
dataset = ImageFolder(root=dataset_path, transform=trans)

<>:1: SyntaxWarning: invalid escape sequence '\C'
<>:1: SyntaxWarning: invalid escape sequence '\C'
C:\Users\nahyu\AppData\Local\Temp\ipykernel_12868\617455154.py:1: SyntaxWarning: invalid escape sequence '\C'
  dataset_path = "F:\프로젝트\CarLogoDetection\data"


In [6]:
train_data, test_data = train_test_split(dataset, test_size = 0.2,random_state = 42)

In [7]:
train_loader = torch.utils.data.DataLoader(train_data,batch_size=batch_size, shuffle=True, num_workers=2, drop_last=True)
teat_loader = torch.utils.data.DataLoader(test_data,batch_size=batch_size, shuffle=False, num_workers=2, drop_last=True)

In [8]:
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, drop_last=True)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False, drop_last=True)

In [9]:
#model

# the resnet34 model
class ResNet34(nn.Module):
    def __init__(self, pretrained):
        super(ResNet34, self).__init__()
        if pretrained is True:
            self.model = pretrainedmodels.__dict__['resnet34'](pretrained='imagenet')
        else:
            self.model = pretrainedmodels.__dict__['resnet34'](pretrained=None)

        # change the classification layer
        self.l0 = nn.Linear(512, class_num)
        self.dropout = nn.Dropout2d(0.6)
    def forward(self, x):
        # get the batch size only, ignore (c, h, w)
        batch, _, _, _ = x.shape
        x = self.model.features(x)
        x = F.adaptive_avg_pool2d(x, 1).reshape(batch, -1)
        x = self.dropout(x)
        l0 = self.l0(x)
        return l0
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model = ResNet34(pretrained=True).to(device)

C:\Users\nahyu\anaconda3\envs\CarLogoDetection\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\nahyu\anaconda3\envs\CarLogoDetection\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [10]:
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [11]:
#train
for i in range(num_epoch):
    for j,[image,label] in enumerate(train_loader):
        x = image.to(device)
        y_= label.to(device)

        optimizer.zero_grad()
        output = model.forward(x)
        loss = loss_func(output,y_)
        loss.backward()
        optimizer.step()

    if i % 10 == 0:
        print(loss)

C:\Users\nahyu\anaconda3\envs\CarLogoDetection\Lib\site-packages\torch\nn\functional.py:1538: UserWarning: dropout2d: Received a 2-D input to dropout2d, which is deprecated and will result in an error in a future release. To retain the behavior and silence this warning, please use dropout instead. Note that dropout2d exists to provide channel-wise dropout on inputs with 2 spatial dimensions, a channel dimension, and an optional batch dimension (i.e. 3D or 4D inputs).
  warnings.warn(warn_msg)


tensor(1.2845, grad_fn=<NllLossBackward0>)
tensor(0.0187, grad_fn=<NllLossBackward0>)
tensor(0.0065, grad_fn=<NllLossBackward0>)
tensor(0.0047, grad_fn=<NllLossBackward0>)
tensor(0.0019, grad_fn=<NllLossBackward0>)
tensor(0.0013, grad_fn=<NllLossBackward0>)
tensor(0.0009, grad_fn=<NllLossBackward0>)
tensor(0.0006, grad_fn=<NllLossBackward0>)
tensor(0.0018, grad_fn=<NllLossBackward0>)
tensor(0.0006, grad_fn=<NllLossBackward0>)


In [12]:
#test
correct = 0
total = 0

with torch.no_grad():
  for image,label in test_loader:
      x = image.to(device)
      y_= label.to(device)
      output = model.forward(x)
      _,output_index = torch.max(output,1)
      total += label.size(0)
      correct += (output_index == y_).sum().float()

  print("Accuracy of Test Data: {}".format(100*correct/total))

Accuracy of Test Data: 100.0


In [13]:
torch.save(model.state_dict(), 'model')

In [ ]:
import torch
import PIL
# 모델 불러오기
model = ResNet34(pretrained=True).to(device)
model.load_state_dict(torch.load("model"))  #학습한 모델 사용

# 새로운 이미지 로드 및 전처리
image_path = "/content/tester.png"  # 사용할 이미지 파일 업로드 후
image = PIL.Image.open(image_path).convert("RGB")
input_image = trans(image).unsqueeze(0)

if torch.cuda.is_available():
    input_image = input_image.to('cuda')

# 모델 입력 및 예측
output = model(input_image)

# 예측 결과 확인
predicted_label = torch.argmax(output)

print("예측된 라벨:", predicted_label)

FileNotFoundError: ignored

In [ ]:
predicted_label.cpu()
k = predicted_label.item()

In [ ]:
com = class_list[k]

print("이 차량 정보: " ,com[0],"사의 차량으로", "최대",com[1],"원 입니다.")